<a href="https://colab.research.google.com/github/OSGeoLabBp/tutorials/blob/master/hungarian/python/simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pythonic kód készítés

Ebben a kis oktatóanyagban egyszerű matematikai problémák Python megoldását mutatjuk be.

##Legnagyobb közös osztó

Találjuk meg két szám legnagyobb közös osztóját. Egy naív megoldás, ha a két számot az összes lehetséges számmal elosztjuk és az osztási maradékot vizsgáljuk. A két szám közül kisebbnél nagyobb közös osztó nem lehet. A hatékonyság érdekében a vizsgálatot a nagyobb számoktól lefelé végezzük. Az alábbi függvény egy lehetséges megoldása a problémának:

In [4]:
def gcd_naiv(a, b):
    """ legnagyobb közös osztó keresés """
    for i in range(min(a, b), 1, -1):
        if a % i == 0 and b % i == 0:
            return i
    return 1

A példában a *range* függvényt használtuk a lehetséges osztók előállítására, csökkenő sorrendben. Például a range(5, 1, -1) a következő sorozatot állítja elő: [5, 4, 3, 2]. A % operátor két egész érték osztási maradékát számítja. Az első olyan szám megtalálásakor, melynek a két bejövő értékkel az osztási maradéka nulla, megvan a megoldást (mivel csökkenő sorrendben haladunk). Amennyiben a ciklusunk végéig nem találunk mindkét számhoz osztót, akkor 1 értéket ad vissza a függvényünk.

Próbáljuk ki a fenti kódot.

In [5]:
gcd_naiv(32, 80)

16

Miért neveztük naívnak a kódot? Nagy számok esetén lassú ez a megoldás. Már Euklidesz talált egy hatékonyabb rekurzív algoritmust:

gcd(a, b) = gcd(b, a % b)

gcd(a, 0) = a

Nézzük meg a megoldás lépéseit a 32 és 80 számok esetén:

| iteráció |    a  |   b   |
-----------|-------|--------
      0    |   80  |   32
      1    |   32  |   16
      2    |   16  |    0

Vegyük észre, hogy a naív megoldásban 32-től egyesével léptünk lefelé a 16-ig, azaz 16-szor futott le a ciklusmag.

A fenti rekurzív algoritmus Python megvalósítása a következő lehet:

In [6]:
def gcd_rec(a, b):
    """ Legnagyobb közös osztó keresés Eulidesz rekurzív algoritmusa """
    if b == 0:
        return a
    return gcd_rec(b, a % b)

Próbáljuk ki a megoldásunkat!

In [8]:
gcd_rec(80, 32)

16

Rekurzióról akkor beszélünk, ha egy függvény közvetlenül vagy közvetve önmagát hívja. A rekurzió általában rövid és könnyen áttekinhető kódot eredményez, de sok memóriát használhat.

Nézzünk meg egy nem rekurzív megoldást:

In [7]:
def gcd_nrec(a, b):
    """ Legnagyobb közös osztó keresés Eulidesz nem rekurzív algoritmusa """
    while b:
        a, b = b, a % b
    return a

Ez is meglehetősen rövid kód. Két részletre térnénk ki itt. a *while b:* sor azt jelenti,hogy a ciklusunkat addig hajtsuk végre, amíg *b* értéke *nulla* nem lesz (egyenértékű a *while b != 0* sorral). Ez abból következik, hogy az egész számok logikai értékként is lehet használni. A nulla a hamis logikai értéket jelenti, minden más érték az igaz logikai értéket. A cikluson belüli sor egyidőben két változó értékét változtatja meg, például Pythonban két változó értékét egysoros utasításssal megvalósíthatjuk: 

*x,y = y, x*

Próbálja ki a fenti kódot:

In [9]:
gcd_nrec(80, 32)

16

A Pythonhoz tartozó modulok sokféle algoritmus kódját tartalmazza, a *math* modulban van egy *gcd* függvény.

In [11]:
from math import gcd
gcd(80, 32)

16

A fenti példa arra is rámutat, hogy a programozási proglémánk megoldására először keressünk kész megoldást, ha nem találunk ilyet, akkor keressünk kidolgozott hatékony algoritmust, melyet megprogramozunk. A saját algoritmus kidolgozásába csak az előző két lehetőség hiányába fogjunk hozzá.

A *timeit* modul segítségével hasonlítsuk össze a fent használt négy függvény futási ideit.

In [19]:
from timeit import timeit

n = 3403
m = 913

print(f"legnagyobb osztó: {gcd(n, m)}")
print(f"naív         : {timeit('gcd_naiv(n, m)', globals=globals(), number=100_000):.2f} másodperc")
print(f"rekurzív     : {timeit('gcd_rec(n, m)', globals=globals(), number=100_000):.2f} másodperc")
print(f"nem rekurzív : {timeit('gcd_nrec(n, m)', globals=globals(), number=100_000):.2f} másodperc")
print(f"math modul   : {timeit('gcd(n, m)', globals=globals(), number=100_000):.2f} másodperc")

legnagyobb osztó: 83
naív         : 4.82 másodperc
rekurzív     : 0.05 másodperc
nem rekurzív : 0.03 másodperc
math modul   : 0.01 másodperc


Hogyan lehetséges, hogy a *math* modul függvénye háromszor-ötször gyorsabb a mi megoldásunknál? A *math* modul függvényét C/C++ nyelven készítették, mely jóval hatékonyabb kódot eredményez mnt a Python.

Változtassa meg az *n* és *m* változók értékét és nézze meg hogyan változnak a futási idők!

##Egész számok átváltása római számmá

A következő példában a római és arab számok közötti átváltást oldjuk meg. Itt arra törekszünk, hogy a lehető legkevesebb feltételes utasítással oldjuk meg a feladatot. Először az arab számok rómaivá alakításával foglalkozzunk. A feladat megoldásához egy segédlistát hozunk létre, mely egyrészt az egy jegyű (egy-három között ismételhető) római számok értékét illetve a speciális két betűből állókat tartalmazza (amikor az első szám kisebb mint az utánakövetkező pl. IV, CD, stb.) valamint nekik megfelelő arab számokat, az arabszámok csökkenő sorrendjében.

In [25]:
roman = ((1000, "M"), (900, "CM"), (500, "D"), (400, "CD"),
         (100, "C"), (90, "XC"), (50, "L"), (40, "XL"),
         (10, "X"), (9, "IX"), (5, "V"), (4, "IV"), (1, "I"))

Ezután már "csak" a fenti listán kell végigmenni és amíg az átváltandó szám értéke nagyobb mint a roman listában az egész szám az eredmény szöveglánchoz hozzá kell adni a római számot és az eredeti számot az értékével csökkenteni. A megoldást egy Python függvénybe helyezzük el:

In [26]:
def toroman(n):
    """ egész szám átváltása római számmá pl. 456 -> CDLVI """
    result = ""
    for item in roman:
        while n >= item[0]:
            result += item[1]
            n -= item[0]
    return result

Ennyi az egész. A megoldás menetét az alábbi táblázattal mutatjuk be:

item     | n   | result
---------|-----|-------
1000,"M" | 456 | -
900,"CM" | 456 | -
500,"D"  | 456 | -
400,"CD" |  56 | **"CD"**
100,"C"  |  56 | "CD"
90,"XC"  |  56 | "CD"
50,"L"   |   6 | **"CDL"**
40,"XL"  |   6 | "CDL"
10,"X"   |   6 | "CDL"
9,"IX"   |   6 | "CDL"
5,"V"    |   1 | **"CDLV"**
4,"IV"   |   1 | "CDLV"
1,"I"    |   0 | **"CDLVI"**

Vigyázat a fenti megoldás 4000 vagy annál nagyobb számokra nem működik. Ezen számok ábrázolása nem lehetséges római számokkal.

Az alábbi kóddal több számra kipróbáljuk az átváltást. Vegye észre, hogy az 5432 esetén az eredmény nem megfelelő (háromnál több azonos betű egymás mellett).

In [30]:
numbers = (3456, 999, 3999, 735, 5432)
for n in numbers:
    print(f"{n:5d}: {toroman(n)}")

 3456: MMMCDLVI
  999: CMXCIX
 3999: MMMCMXCIX
  735: DCCXXXV
 5432: MMMMMCDXXXII


Próbáljuk meg megoldani az átváltást visszafelé is. Itt a logikánk az lesz, hogy először váltsuk át a speciális két betűből álló részeket, ahol az első betű számértéke kisebb mint az őt követő (pl. XC, CM, stb.). A római számoknál nem létezik helyiérték így nem kell az elején vagy a végén kezdeni az átváltást. Ismét egy segéd listát és egy segéd szótárat hozunk létre az algoritmus egyszerűsítésére:

In [31]:
arabic = {'IV': 4, 'IX': 9, 'XL': 40, 'XC': 90, 'CD': 400,
          'CM': 900, 'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100,
          'D': 500, 'M': 1000}
keys = tuple(arabic.keys())
keys
#('IV', 'IX', 'XL', 'XC', 'CD', 'CM', 'I', 'V', 'X', 'L', 'C', 'D', 'M')

('IV', 'IX', 'XL', 'XC', 'CD', 'CM', 'I', 'V', 'X', 'L', 'C', 'D', 'M')

Vegyük észre, hogy a *keys* listában elől vannak a kétbetűből álló speciális esetek. Ezek után a *key* értékeket a megadott sorrendben meg kell keresnünk az átalakítandó római számban és az arab értékeiket összegezni egy változóban.

In [32]:
def toint(s):
    """ Convert roman number to int """
    result = 0
    s = s.upper()   # nagybetűssé konvertálás
    for key in keys:    # végigmegyünk a kulcsokon
        while key in s:
            result += arabic[key]
            s = s.replace(key, "", 1)
    return result

A fenti függvényben a *replace* szövegláncokra alkalmazható függvényt egy előfordulás cseréjére használjuk, ezt jelenti a harmadik paraméterben az egyes. A *replace* függvény az összes előfordulást lecseréli alapértelmezésben, ami nekünk nem lenne jó most.

Próbáljuk ki néhány számon!

In [34]:
romans = ("MMMCDLVI", "CMXCIX", "MMMCMXCIX", "DCCXXXV", "MMMMMCDXXXII")
for r in romans:
    print(f"{r:>12s}: {toint(r):5d}")

    MMMCDLVI:  3456
      CMXCIX:   999
   MMMCMXCIX:  3999
     DCCXXXV:   735
MMMMMCDXXXII:  5432


Vegyük észre, hogy az *arabic szótárból* a *roman* lista is előállítható:



In [42]:
tuple(sorted(list(zip(arabic.values(), arabic.keys())))[::-1])

((1000, 'M'),
 (900, 'CM'),
 (500, 'D'),
 (400, 'CD'),
 (100, 'C'),
 (90, 'XC'),
 (50, 'L'),
 (40, 'XL'),
 (10, 'X'),
 (9, 'IX'),
 (5, 'V'),
 (4, 'IV'),
 (1, 'I'))

##Hurkok keresése egy irányítatlan gráfban

Egy gráf leírását az éleket tartalmazó fájllal vagy listával adjuk meg. A gráfra kikötjük hogy két csomópont között csak egy közvetlen kapcsolat lehet. Az éleket a kezdő és vég csomópont azonosítóival írjuk le, emellett az él hosszát és a végpontok magasságkülönbségét tároljuk (szintezési hálózat).

```
1 +-------+2            Élek listája
  |      /|        kezdő vég   táv   dm
  |     / |          1    2     1    0.511
  |    /  |          2    3     2   -0.190
  |   /   |          3    4     1    0.072
  |  /    |          4    1     2   -0.392
  | /     |          4    2     2.5  0.120
  |/      |
 4+-------+3

```

A hurkok keresését először a mélységben lépő algoritmussal valósítottuk meg. Az alábbi kódot például arra használhatjuk, hogy egy szintezési hálózatban a körzárásokat ellenőrizzük.

A mélységben először kereső algoritmus azt jelenti, hogy a gráfot egy adott éltől kezdődően egy fába fejtjük ki, oly módon, hogy a fa ágain ugyanaz a csomópont ne ismétlődhessen meg.

A hurok keresést minden egyes élre végrehajtjuk. A kezdeti élt egyesével növeljük a csatlakozó élek közül (addp függvény). Egy hurok megtalálása esetén vizsgáljuk, hogy más élből indulva megtaláltuk-e már ezt. Két hurkot a benne lévő csomópontok száma és a csomópontokból alkotott halmazok összehasonlításával valósitjuk meg. Például a [1, 2, 4, 1] és a [4, 2, 1, 4] hurkok hossza (4) azonos és a belőlük alkotott halmazok {1, 2, 4} is azonosak.

Végül a hosszuk alapján rendezzük a hurkokat, összegezzük a hosszakat és a magasságkülönbségeket és kiírjuk az eredményt.

In [67]:
# példa adatállomány előállítása
edges_dic = {(1,2): [1, 0.511],
             (2,3): [2,-0.190],
             (3,4): [1, 0.072],
             (4,1): [2,-0.392],
             (4,2): [2.5,0.120]}
edges = list(edges_dic.keys())
edges

[(1, 2), (2, 3), (3, 4), (4, 1), (4, 2)]

In [66]:
def addp(edges, loop, indx):
    """ find and add a point to the loop
        :param edges: edges of graph
        :param loop: actual loop
        :param indx: edge indices

        :returns: extended loop and new indx as a tuple
    """
    for i in range(indx[-1], len(edges)):
        edge = edges[i]
        if edge[0] == loop[-1] and edge[1] != loop[-2] and edge[1] not in loop[1:]:
            # connection to start point
            loop.append(edge[1])
            indx[-1] = i
            indx.append(0)
            return loop, indx
        if edge[1] == loop[-1] and edge[0] != loop[-2] and edge[0] not in loop[1:]:
            # connection to end point
            loop.append(edge[0])
            indx[-1] = i
            indx.append(0)
            return loop, indx
    return loop, indx

# find all unique loops
loops = []
for edge in edges:
    loop = [edge[0], edge[1]]
    indx = [0, 0]
    while True:
        n = len(loop)
        loop, indx = addp(edges, loop, indx)
        if loop[0] == loop[-1]:     # closed loop found
            n1 = len(loop)
            s1 = set(loop)
            for loop2 in loops:     # check same loop found?
                if n1 == len(loop2) and s1 == set(loop2):
                    break
            else:
                loops.append(loop[:])   # make a copy of list
            n = len(loop)           # force back step
        if len(loop) == n:
            # no new point or loop found step back
            loop.pop()
            indx.pop()
            indx[-1] += 1
            if len(loop) < 2:       # no more step back
                break
loops.sort(key=len)                 # sort loop by length
n = 0
m = len(loops[-1])                  # length of longest loop
# sum up distances and values
print("Zárás    Hossz Hurok")
for i, loop1 in enumerate(loops):
    n += 1
    # calculate sum distance and value
    sdist = 0
    sdm = 0
    last = loop1[0]                 # start point
    for node in loop1[1:]:
        indx = last,node
        if indx in edges_dic:       # forward direction
            sdist += edges_dic[indx][0]
            sdm += edges_dic[indx][1]
        else:                       # reverse direction
            indx = node,last
            sdist += edges_dic[indx][0]
            sdm -= edges_dic[indx][1]
        last = node
    print (f"{sdm:.3f} {sdist:8.1f} {loop1}")
print(f"Hurkok száma: {n}, Max hurok hossz: {m}")

Zárás    Hossz Hurok
-0.001      5.5 [1, 2, 4, 1]
0.002      5.5 [2, 3, 4, 2]
0.001      6.0 [1, 2, 3, 4, 1]
Hurkok száma: 3, Max hurok hossz: 5
